# GIS2RHESSys
**Software / computer requirements:**
- Max OS / LINUX (e.g., Urbantu) 
- GRASS 7.4.x installed 
- R 3.6.x or above
- R package “rgrass7” and its dependences


## Breakdowns of the workflow shell script

<div class="alert alert-block alert-danger">
This page serves as a document embedded with code blocks for demostrations.
This is not a JupyterNotebook for users to execute codes listed below. 
Users may copy/paste the codes to assemble a workflow bash/shell script running on local machine.
</div>

### 1. Initiate project directory and set up GRASS GIS dataset for RHESSys modeling  

1.1 **Set system paths to GRASS, R, and Github libraries**

<div class="alert alert-block alert-info">
The PATH settings below are for MAC OS. Users may need to modify some paths for the LINUX environment.
</div>

In [ ]:
# path setup below is for Mac OS
# adding R paths
export PATH=$PATH:/usr/local/bin
export PATH=$PATH:/Library/Frameworks/R.framework/Resources
# adding GRASS bin paths
export PATH=$PATH:/Applications/GRASS-7.4.4.app/Contents/Resources/bin:/Applications/GRASS-7.4.4.app/Contents/Resources/scripts:/etc:/usr/lib
# adding GRASS environment variables: GISBASE_USER and GISBASE_SYSTEM in Grass.sh
export PATH=$PATH:~/Library/GRASS/7.4/Modules/bin:~/Library/GRASS/7.4/Modules/scripts
# adding GDAL and EPSG code lookup paths (Running gdal-config --datadir shows where GDAL searches for gcs.csv)
export GDAL_DATA=/Applications/GRASS-7.4.4.app/Contents/Resources/share/gdal
GITHUBLIBRARIES="https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/libraries"

1.2 **User inputs for project directory, RHESSys model name, GIS projection and resolution**

<div class="alert alert-block alert-info">
Enter the full path to the project directory (does not need to be created first; see the next step).<br>
When search for UTM EPSG code, we recommend to use datum NAD83 for the U.S. catchments.<br>
The spatial resolution (e.g., 3/10/30 m) of the elevation raster is often the spatial resolution of the RHESSys model.
</div>

In [ ]:
PROJDIR='/Workspace/current_projects/Delight' # full path to the project location;
EPSGCODE='EPSG:26918' # EPSG code for NAD83 UTM ##N for the catchment
RESOLUTION=10 #spatial resolution (meters) of the grids
RHESSysNAME='rhessys' # e.g., rhessys
GISDBASE="$PROJDIR"/grass_dataset
LOCATION_NAME="$RHESSysNAME"
LOCATION="$GISDBASE"/$LOCATION_NAME
MAPSET=PERMANENT

<div class="alert alert-block alert-info">
Create project directory structures and RHESSys model directory structures. We only do this once initial although these commands do not overwrite existing directories.
</div>

In [ ]:
### ... create project folder structures
mkdir "$PROJDIR"
mkdir "$PROJDIR"/grass_dataset
mkdir "$PROJDIR"/raw_data
### ... create rhessys folder structures
mkdir "$PROJDIR"/"$RHESSysNAME"
mkdir "$PROJDIR"/"$RHESSysNAME"/defs
mkdir "$PROJDIR"/"$RHESSysNAME"/flows
mkdir "$PROJDIR"/"$RHESSysNAME"/worldfiles
mkdir "$PROJDIR"/"$RHESSysNAME"/clim
mkdir "$PROJDIR"/"$RHESSysNAME"/tecfiles
mkdir "$PROJDIR"/"$RHESSysNAME"/output

<div class="alert alert-block alert-info">
Create project GIS database. We only do this once initial although this command does not overwrite existing database.
</div>

In [ ]:
### ... create grass database
grass74 -c $EPSGCODE -e "$LOCATION" # it spits out ERROR for location not available so that it can create new GIS dataset (just ignore this error).


1.3 **Import and process of elevation data**

<div class="alert alert-block alert-info">
set path to the elevation raster file (e.g., "$PROJDIR"/raw_data/DEM1m.tif)
</div>

In [ ]:
downloadedDEMfile="$PROJDIR"/raw_data/DEM1m.tif

<div class="alert alert-block alert-info">
(Option 1) Assume that we need to change the elevation raster projection and resample into desired spatial resolution.<br> a) We ask GRASS to read in the elevation raster source and then resample the spatial scale<br> b) We export the sampled elevation information into a new raster "demXXm.tif", where XX is the spatial resoluted defined above<br> c) We import the "demXXm.tif" to the project GIS database with reprojection process (based on gdal) and set the project GIS database spatial extension and resolution to match the "demXXm.tif".
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -e --overwrite input="$downloadedDEMfile" output=demRAW location=elevationRAW
LOCATIONDEM="$GISDBASE"/elevationRAW
grass74 "$LOCATIONDEM"/$MAPSET --exec g.region raster=demRAW
grass74 "$LOCATIONDEM"/$MAPSET --exec g.region res=$RESOLUTION -a -p
grass74 "$LOCATIONDEM"/$MAPSET --exec r.resamp.stats -w input=demRAW output=dem$RESOLUTION'm' ### skip this if no resampling spatial scale
grass74 "$LOCATIONDEM"/$MAPSET --exec r.out.gdal --overwrite input=dem$RESOLUTION'm' output="$PROJDIR"/raw_data/dem$RESOLUTION'm.tif' format=GTiff
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -o -e --overwrite input="$PROJDIR"/raw_data/dem$RESOLUTION'm.tif' output=dem
grass74 "$LOCATION"/$MAPSET --exec g.region raster=dem
rm -rf "$LOCATIONDEM"

<div class="alert alert-block alert-info">
(Option 2) If the source elevation raster has the correct projection (UTM) and resolution, then we can just simply import it to the project GIS dataset and set the project GIS database spatial extension and resolution to match the elevation raster.
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -o -e --overwrite input="$PROJDIR"/raw_data/dem$RESOLUTION'm.tif' output=dem
grass74 "$LOCATION"/$MAPSET --exec g.region raster=dem

1.4 **Define catchment outlet**

<div class="alert alert-block alert-info">
(Option 1) Use Lat/Long (WSG84 decimal degree) to define catchment outlet. Note that keep the negative sign if applied.
</div>

In [ ]:
gageLat='39.47947' # catchment outlet WSG84 Lat (decimal degree)
gageLong='-76.67803' # catchment outlet WSG84 Long (decimal degree; includes the negative sign if applied)
eval $(grass74 "$LOCATION"/$MAPSET --exec m.proj -i coordinates=$gageLong,$gageLat separator=space | awk '{print "xyCoord=" $1 "," $2}')
echo $xyCoord | grass74 "$LOCATION"/$MAPSET --exec v.in.ascii in=- out=outlet x=1 y=2 separator=, --overwrite

<div class="alert alert-block alert-info">
(Option 2) Import a catchment outlet shapefile. Note that this shapefile should contain only ONE point.
</div>

In [ ]:
gageShapefile="$PROJDIR"/raw_data/USGS_Delight.shp
grass74 "$LOCATION"/$MAPSET --exec v.in.ogr --overwrite input="$gageShapefile" output=outlet location=outletRAW
LOCATIONOUTLET="$GISDBASE"'/outletRAW'
grass74 "$LOCATION"/$MAPSET --exec v.proj --overwrite location=outletRAW mapset=PERMANENT input=outlet output=outlet
rm -rf "$LOCATIONOUTLET"

1.5 **Catchment delineation**

<div class="alert alert-block alert-info">
User needs to provide an estimation of catchment area (m^2) and threshold values to define modeling stream network. <br>
The threshold value is an area (m^2), e.g., 620000 m^2 = 62 ha, that defines the extension of a major stream channels for RHESSys; it also affects the hillslope and subcatchment structures. <br>
The second threshold value is also an area, e.g., 100000 m^2 = 10 ha, that defines the full extension of a stream network, including head water channels, for riparian delineation and surface water routing.
</div>

In [ ]:
expectedDrainageArea=10956000 # meter sq.
expectedThresholdModelStr=620000 # meter sq.
expectedThresholdStrExt=100000 # meter sq.

In [ ]:
GRASS_thres=$(($expectedThresholdModelStr/$RESOLUTION/$RESOLUTION)) # grid cell for stream network and hillslope configuration
GRASS_thresII=$(($expectedThresholdStrExt/$RESOLUTION/$RESOLUTION))
GRASS_drainarea_lowerbound=$((98*$expectedDrainageArea/$RESOLUTION/$RESOLUTION/100)) # (allow 2% error)
GRASS_drainarea_upperbound=$((102*$expectedDrainageArea/$RESOLUTION/$RESOLUTION/100)) # (allow 2% error)

grass74 "$LOCATION"/$MAPSET --exec bash <(curl -s "$GITHUBLIBRARIES"/grass_delineation_1.sh) $GRASS_thres $GRASS_drainarea_lowerbound $GRASS_drainarea_upperbound
grass74 "$LOCATION"/$MAPSET --exec Rscript <(curl -s "$GITHUBLIBRARIES"/basin_determine.R)
grass74 "$LOCATION"/$MAPSET --exec r.watershed elevation=dem threshold=$GRASS_thresII stream=strExt --overwrite # full stream extension;
grass74 "$LOCATION"/$MAPSET --exec bash <(curl -s "$GITHUBLIBRARIES"/grass_delineation_2.sh)

<div class="alert alert-block alert-info">
When this procedure is completed, a catchment is delineated, along with slope, aspect, wettness index (D-inf), east & west horizon sun angles, hillslope IDs, subcatchment IDs, patch IDs, streams, flow accumulation, and drainage direction (D-Inf). GIS computation region and mask are also set by the delineated catchment. 
</div>

1.6 **Define climate zones**

<div class="alert alert-block alert-info">
(Option 1) The coarse climate zone setting  
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="zone_hill = hill"

<div class="alert alert-block alert-info">
(Option 2) Using statistical cluster analysis, we delineate the elevation bands and further partition the bands by aspect, slope, and hillslopes. The resulting climate zone is finer than the coarse setting (option 1) but not as fine as the finest setting (option 3).
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec Rscript <(curl -s "$GITHUBLIBRARIES"/zone_cluster.R) dem slope aspect hill zone_cluster

<div class="alert alert-block alert-info">
(Option 3) The finest climate zone setting, same as elevation resolution. This also increases the RHESSys computation time
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="zone_patch = patch"

1.7 **Define isohyet (MUST)**

<div class="alert alert-block alert-info">
(Option 1) import ioshyet raster
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -o --overwrite input="$PROJDIR"/raw_data/isohyet.tif output=isohyet

<div class="alert alert-block alert-info">
(Option 2) calculate isohyet by raster calculator (the calculation below is just an example)
</div>

In [ ]:
grass74 $LOCATION/$MAPSET --exec r.mapcalc --overwrite expression="isohyet = (1 +(4.268e-04)*(dem-1200) +(-6.799e-05)*slope +(5.845e-03)*sin(aspect*0.01745329)*sin(slope*0.01745329) +(-1.267e-02)*cos(aspect*0.01745329)*sin(slope*0.01745329))*1.45817781"

<div class="alert alert-block alert-info">
(Option 3) no ioshyet --> set ioshyet to one everywhere
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="isohyet = 1"

1.8 **Import SSURGO soil information**

<div class="alert alert-block alert-info">
This step requires the SSURGO spatial and tabular dataset. <br>
Users need to set the path to the SSURGO dataset, e.g., "$PROJDIR"/raw_data/MD005 <br>
The procedure below reprojects the SSURGO mukey (areal polygon) shapefile onto the project GIS database, extract soil information from the tabular dataset, and map the extracted soil information to the corresponding mukey polygons.
</div>

In [ ]:
downloadedSSURGOdirectoryPATH="$PROJDIR"/raw_data/MD005
grass74 "$LOCATION"/$MAPSET --exec v.in.ogr --overwrite input="$downloadedSSURGOdirectoryPATH"/spatial/soilmu_a_"$(echo ${downloadedSSURGOdirectoryPATH##*/} | tr '[A-Z]' '[a-z]')".shp output=ssurgo location=soilRAW
LOCATIONSOIL="$GISDBASE"/soilRAW
### ... import (reprojected) soil data to ""$LOCATION"/$MAPSET" and convert shape-polygon to raster soil (USDA) classes
grass74 "$LOCATION"/$MAPSET --exec v.proj --overwrite location=soilRAW mapset=PERMANENT input=ssurgo output=ssurgo
grass74 "$LOCATION"/$MAPSET --exec v.to.rast --overwrite input=ssurgo use=cat output=soil_ssurgo
grass74 "$LOCATION"/$MAPSET --exec v.db.select map=ssurgo separator=comma file="$PROJDIR"/"$RHESSysNAME"/soil_cat_mukey.csv
Rscript <(curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_extraction.R) "$downloadedSSURGOdirectoryPATH"
grass74 "$LOCATION"/$MAPSET --exec Rscript <(curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_soiltexture2gis.R) "$PROJDIR"/"$RHESSysNAME"/soil_cat_mukey.csv "$downloadedSSURGOdirectoryPATH"/soil_mukey_texture.csv
rm -rf "$LOCATIONSOIL"

1.9 **Import high resolution LULC data and aggregate information into modeling patch (elevation grids)**

<div class="alert alert-block alert-info">
For any LULC data, we breakdown the LULC information by the following compositions at the patch level:<br>
- forest composition<br>
- shrub composition<br>
- crop composition<br>
- grass composition<br>
- imperviousness composition<br>
<br>
For imperviousness compositional fraction, we further partition this fraction into different catagories:<br>
- impervious structure roof<br>
- impervious parking/non-structural surface <br>
- impervious (paved) road <br>
<br>
the code block below extracts the fractions of LULC classification in each modeling patch
</div>

In [ ]:
downloadedLULCfile="$PROJDIR"/raw_data/'LULC1m.tif' 
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -e --overwrite input="$downloadedLULCfile" output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
grass74 "$LOCATION"/$MAPSET --exec r.to.vect input=patch output=patch type=area
grass74 "$LOCATIONLULC"/$MAPSET --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
grass74 "$LOCATIONLULC"/$MAPSET --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
grass74 "$LOCATIONLULC"/$MAPSET --exec Rscript <(curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R) patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'

<div class="alert alert-block alert-info">
To translate LULC classifications into the compositions above, we need to define each classification class by the the compositions.
</div>

For example, NLCD code 42 - deciduous forest, 22 low-density urban, and 24 high-density urban,

| classification code | forestComp | shrubComp | CropComp | grassComp | impComp |
|------|------|------|------|------|
|   42 | 1| 0| 0| 0| 0|
|   22 | 0.2| 0| 0| 0.3|0.5 |
|   24 | 0| 0| 0| 0| 1|

<div class="alert alert-block alert-info">
(Option 1) This block of code below is to create an excel table (just like the example above) for users to fill in the information when LULC classification is not one of the standard ones, e.g., NLCD, ...
</div>

In [ ]:
Rscript <(curl -s "$GITHUBLIBRARIES"/LULC_codeinformation.R) "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' "$PROJDIR"/"$RHESSysNAME"/lulc_codeinformation.csv
grass74 "$LOCATION"/$MAPSET --exec Rscript <(curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R) patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' "$PROJDIR"/"$RHESSysNAME"/lulc_codeinformation.csv
rm -rf "$LOCATIONLULC"

<div class="alert alert-block alert-info">
(Option 2) NLCD LULC classification
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec Rscript <(curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R) patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' <(curl -s https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_30m_NLCD.csv)
rm -rf "$LOCATIONLULC"

<div class="alert alert-block alert-info">
(Option 3) Baltimore LULC classification
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec Rscript <(curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R) patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' <(curl -s https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_1m_Chesapeake_Conservancy.csv)
rm -rf "$LOCATIONLULC"

<div class="alert alert-block alert-info">
(Option 4) Virginia LULC classification
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec Rscript <(curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R) patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' <(curl -s https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_1m_va.csv)
rm -rf "$LOCATIONLULC"

1.10 **Define vegetation for vegetated LULC areas**

<div class="alert alert-block alert-info">
The forest/shrub/crop/grass LULC composition calculated above is a defined area for forest/shrub/crop/grass, and we have to define what vegetations should be growing in the area. Current script can support up to 15 vegetation types per patch.
</div>

<div class="alert alert-block alert-info">
For example, we model an oak-dominated canopy in a forested catchment. <br> <br>
We determine the locations of oak canopy (the first line) <br>
We simply model the full canopy as oak and maple (the second line); "FFrac" indicates the oak composition in the full canopy. In this example, FFrac = 0.8, i.e., we treat the full canopy as 80% oak and 20% maple.<br>
We set the LAI for the oak canopy (the third line). In this example, LAI is a constant 5.5. One can substitude the 5.5 value by an imported remotely sensed LAI raster. 
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="oak_canopy = if(forestFrac>0,102,null())" # - the first line
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="oak_canopy_FFrac = if(forestFrac>0,0.8,null())" # - the second line
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="oak_canopy_LAI = if(forestFrac>0,5.5,null())" # - the third line

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="maple_canopy = if(forestFrac>0,111,null())"
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="maple_canopy_FFrac = if(forestFrac>0,0.2,null())"
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="maple_canopy_LAI = if(forestFrac>0,5.5,null())"

<div class="alert alert-block alert-info">
Similarly, we can model the lawn/pasture the same way.
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="grass1StratumID = if(lawnFrac>0,3,null())"
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="grass1FFrac = if(lawnFrac>0,1.0,null())"
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="grass1LAI = if(lawnFrac>0,1.5,null())"

1.11 **Import road network (shapefile; if applied)**

<div class="alert alert-block alert-info">
This is optional to import a vector-based road network.
</div>

In [ ]:
downloadedROADfile="$PROJDIR"/raw_data/'Roads_GDT_MSA.shp'
grass74 "$LOCATION"/$MAPSET --exec v.in.ogr --overwrite input="$downloadedROADfile" output=roads location=roadRAW
LOCATIONROAD="$GISDBASE"/roadRAW
grass74 "$LOCATION"/$MAPSET --exec v.proj --overwrite location=roadRAW mapset=$MAPSET input=roads output=roads
grass74 "$LOCATION"/$MAPSET --exec v.to.rast --overwrite input=roads output=vector_roads use=cat
rm -rf "$LOCATIONROAD"

1.12 **Define riparian (if applied)**

<div class="alert alert-block alert-info">
(Option 1) import a determined riparian raster
</div>

In [ ]:
grass74 "$LOCATION"/"$MAPSET" --exec r.in.gdal -o --overwrite input="$PROJDIR"/raw_data/riparian.tif output=riparian

<div class="alert alert-block alert-info">
(Option 2) Use HANDS tool to define riparian zone. This is timely process.
</div>

In [ ]:
grass74 "$LOCATION"/"$MAPSET" --exec Rscript <(curl -s "$GITHUBLIBRARIES"/elevation_analysis.R) dem colmap rowmap drain hill strExt
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc expression="riparian_hands = if( handsDEM < 5, 1, null())" --overwrite

1.13 **Define surface drainage features for urban catchments (if applied)**

<div class="alert alert-block alert-info">
Current RHESSys EC supports storm drinage along road network, (subsurface) sewer drinage, and surface drainage. 
</div>

In [ ]:
### ... storm drinage along the paved roads
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc expression="roadDEM = if(pavedRoadFrac>0,dem,null())"
grass74 "$LOCATION"/$MAPSET --exec r.watershed -s --overwrite elevation=roadDEM drainage=roadDrain
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="roadExit = if(roadDrain<0, patch, null())"

In [ ]:
### ... assume sewer lines go along major roads (imported from shapefile) and sewer lines drainge a neigbour area
grass74 "$LOCATION"/$MAPSET --exec r.buffer --overwrite input=roads output=roadbuff distances=30
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc expression="sewercover = if( (lawnFrac>0.1 || impFrac >0.1) && pavedRoadFrac<0.3 && roadbuff>0, 1, null())" --overwrite

In [ ]:
### ... additional surface drainage  (other than roof, driveway, parking, and paved roads), e.g., drainage hole in playground / garden
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="addsurfdrain = if( forestFrac<0.5 && roofBuff>0, 1 , null())"

In [ ]:
### ... soil compactness by construction
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="roof = if(roofFrac > 0.5, 1, null())"
grass74 "$LOCATION"/$MAPSET --exec r.buffer --overwrite input=roof output=roofBuff distances=30
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="naturalLand = if( (forestFrac>=1 || lawnFrac>=1)&& isnull(roofBuff), 1 , null())"
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="compactedsoil = if( naturalLand>0, null(), basin)"

2. ### Extract GRASS GIS dataset into RHESSys modeling inputs

2.1 **Build template**

In [ ]:
# path for the template file
templateFile="$PROJDIR"/"$RHESSysNAME"/g2w_template.txt 
#
# set paths for RHESSys input files
# 1 = Yes, output this file; 0 = No, do not output this file
echo outputWorldfile \""$PROJDIR"/"$RHESSysNAME"/worldfiles/worldfile.csv\" 1 > "$templateFile"
echo outputWorldfileHDR \""$PROJDIR"/"$RHESSysNAME"/worldfiles/worldfile.hdr\" 1 >> "$templateFile"
echo outputDefs \""$PROJDIR"/"$RHESSysNAME"/defs\" 1 >> "$templateFile"
echo outputSurfFlow \""$PROJDIR"/"$RHESSysNAME"/flows/surfflow.txt\" 1 >> "$templateFile"
echo outputSubFlow \""$PROJDIR"/"$RHESSysNAME"/flows/subflow.txt\" 1 >> "$templateFile"
#
# set climate station ID and file name
echo stationID 101 >> "$templateFile"
echo stationFile \""clim/Oregon.base"\" >> "$templateFile"
#
# the following maps that must be provided with syntex:
# echo keyword <map> >> "$templateFile"
echo basinMap basin >> "$templateFile"
echo hillslopeMap hill >> "$templateFile"
echo zoneMAP zone_cluster >> "$templateFile"
echo patchMAP patch >> "$templateFile"
echo soilidMAP soil_texture_fill >> "$templateFile"
echo xMAP xmap >> "$templateFile"
echo yMAP ymap >> "$templateFile"
echo demMAP dem >> "$templateFile"
echo slopeMap slope >> "$templateFile"
echo aspectMAP aspect >> "$templateFile"
echo twiMAP wetness_index >> "$templateFile"
echo whorizonMAP west_180 >> "$templateFile"
echo ehorizonMAP east_000 >> "$templateFile"
echo isohyetMAP isohyet >> "$templateFile"
echo rowMap rowmap >> "$templateFile"
echo colMap colmap >> "$templateFile"
echo drainMap drain >> "$templateFile"
# ... impervious and its breakdown
echo impFracMAP impFrac >> "$templateFile"
echo roofMAP roofFrac >> "$templateFile"
echo drivewayMAP drivewayFrac >> "$templateFile"
echo pavedRoadFracMAP pavedRoadFrac >> "$templateFile"
# ... forest vegetations
# ... ... vegetation 1
echo forestFracMAP forestFrac_manipu1 >> "$templateFile"
echo tree1StratumID forestStratumID >> "$templateFile"
echo tree1FFrac forestFFrac_manipu1 >> "$templateFile"
echo tree1LAI forestLAI >> "$templateFile"
# ... ... vegetation 2
echo tree2StratumID evergreenBGCStratumID >> "$templateFile"
echo tree2FFrac smallPineFFrac >> "$templateFile"
echo tree2LAI smallPineLAI >> "$templateFile"
# ... shrub vegetation
echo shrubFracMAP shrubFrac >> "$templateFile"
echo shrub1StratumID shrub1StratumID >> "$templateFile"
echo shrub1FFrac shrub1FFrac >> "$templateFile"
echo shrub1LAI shrub1LAI >> "$templateFile"
# ... crop vegetation
echo cropFracMAP cropFrac_manipu1 >> "$templateFile"
echo crop1StratumID crop1StratumID >> "$templateFile"
echo crop1FFrac crop1FFrac >> "$templateFile"
echo crop1LAI crop1LAI >> "$templateFile"
# ... lawn/pasture vegetation
echo grassFracMAP lawnFrac_manipu1 >> "$templateFile"
echo grass1StratumID lawn1StratumID >> "$templateFile"
echo grass1FFrac lawn1FFrac >> "$templateFile"
echo grass1LAI lawn1LAI >> "$templateFile"
# ... modeling stream-grids
echo streamMap strExt >> "$templateFile"
#
# The following maps are optional; User can comment out the lines that do not apply using "#" up front.
echo streamFullextension strExt >> "$templateFile"
#echo unpavedroadMap NA >> "$templateFile"
echo riparianMAP riparian_hands >> "$templateFile"
#echo sewercoverMAP sewercover >> "$templateFile"
#echo pipecoverMAP NA >> "$templateFile"
#echo stormdrainMAP roadExit >> "$templateFile"
#echo compactedsoilMAP compactedsoil >> "$templateFile"
#echo additionalSurfaceDrainMAP addsurfdrain >> "$templateFile"

2.2 **Select extraction script for the desired RHESSys version**

<div class="alert alert-block alert-info">
So far we only support the RHESSys EC build. Other builds will come in the near future.
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec Rscript <(curl -s "$GITHUBLIBRARIES"/g2w_cf_RHESSysEC.R) "$PROJDIR" default default default $templateFile
Rscript <(curl -s "$GITHUBLIBRARIES"/LIB_RHESSys_writeTable2World.R) na "$PROJDIR"/"$RHESSysNAME"/worldfiles/worldfile.csv "$PROJDIR"/"$RHESSysNAME"/worldfiles/worldfile